# Bycicle Counter Prediction

In [1]:
# imports
import xgboost as xgb
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import math


# NOTES

- prohibited to select on the month of july

## TO DO

- check for assumptions

## 1. Data gathering

In [ ]:
# download extended bycicle counter datasets (!wget if not on UNIX)
!curl 'https://data.stad.gent/api/explore/v2.1/catalog/datasets/fietstelpaal-coupure-links-2023-gent/exports/csv?lang=en&timezone=Europe%2FBrussels&use_labels=true&delimiter=%3B' -o 'data/fiets_2023.csv'
!curl 'https://data.stad.gent/api/explore/v2.1/catalog/datasets/fietstelpaal-coupure-links-2022-gent/exports/csv?lang=en&timezone=Europe%2FBrussels&use_labels=true&delimiter=%3B' -o 'data/fiets_2022.csv'
!curl 'https://data.stad.gent/api/explore/v2.1/catalog/datasets/fietstelpaal-coupure-links-2021-gent/exports/csv?lang=en&timezone=Europe%2FBrussels&use_labels=true&delimiter=%3B' -o 'data/fiets_2021.csv'

# Download weather data
!curl 'https://archive-api.open-meteo.com/v1/archive?latitude=51.100006&longitude=3.699997&start_date=2021-03-01&end_date=2023-07-31&hourly=temperature_2m,precipitation,rain,snowfall,snow_depth,cloudcover,windspeed_10m,windgusts_10m&format=csv' -o 'data/weather_data_full.csv'

## 2. Preprocessing

In [2]:
# Load in data
fiets_2023 = pd.read_csv('./data/fiets_2023.csv',delimiter=';')
fiets_2022 = pd.read_csv('./data/fiets_2022.csv',delimiter=';')
fiets_2021 = pd.read_csv('./data/fiets_2021.csv',delimiter=';')

weather_data = pd.read_csv('./data/weather_data_full.csv',skiprows=3)
test_data = pd.read_csv('./data/test_data.csv')

# concatenate the training data
train_data = pd.concat([fiets_2021, fiets_2022, fiets_2023], ignore_index=True)

In [3]:
# Convert time columns to pandas datetime format
## test data
test_data['datetime'] = pd.to_datetime(test_data['Date_hour'])

## train data
train_data['datetime'] = pd.to_datetime(train_data['Datum']+ ' ' + train_data['Uur5Minuten'])

## weather data
weather_data['datetime'] = pd.to_datetime(weather_data['time'])

In [4]:
# drop irrelevant time columns
train_data = train_data[['datetime','Totaal']]
weather_data = weather_data.drop('time',axis =1)
test_data = test_data.drop('Date_hour',axis=1)

# Remove datapoints later than 30th of june 2023
train_data = train_data[train_data['datetime'] <= '2023-06-30 23:00']

# Group by hour
train_data = train_data.groupby(train_data['datetime'].dt.strftime('%Y-%m-%d %H:00'))['Totaal'].sum()

# reset index
train_data = train_data.reset_index()

# reset "datetime" column to pandas datetime format as this changes to "object" type during execution of the "groupby" function.
train_data['datetime'] = pd.to_datetime(train_data['datetime'])

In [5]:
# Merge weather data with training data and test data, based on timepoints
train_data = pd.merge(train_data, weather_data, on='datetime', how='inner')
test_data = pd.merge(test_data, weather_data, on='datetime', how='inner')

# index
test_data.set_index('Id')

,datetime,temperature_2m (°C),precipitation (mm),rain (mm),snowfall (cm),snow_depth (m),cloudcover (%),windspeed_10m (km/h),windgusts_10m (km/h)
Id,,,,,,,,,
0,2023-07-01 00:00:00,18.0,0.0,0.0,0.0,0.0,100,18.8,37.1
1,2023-07-01 01:00:00,17.7,0.0,0.0,0.0,0.0,100,20.4,39.6
2,2023-07-01 02:00:00,17.4,0.0,0.0,0.0,0.0,100,21.9,41.0
3,2023-07-01 03:00:00,17.2,0.0,0.0,0.0,0.0,100,21.7,41.8
4,2023-07-01 04:00:00,17.2,0.0,0.0,0.0,0.0,100,20.9,41.8
...,...,...,...,...,...,...,...,...,...
739,2023-07-31 19:00:00,17.6,0.4,0.4,0.0,0.0,100,17.5,30.2
740,2023-07-31 20:00:00,17.3,0.1,0.1,0.0,0.0,98,17.0,31.7
741,2023-07-31 21:00:00,17.0,0.1,0.1,0.0,0.0,82,17.1,32.8


## 3. Data exploration

- Basic info
- Correlation
- PCA is unnessecary as we aren't dealing with a large number of variables

### 3.1 Basic information

In [ ]:
# Check basic statistics and data structure
print(train_data.describe())

### 3.2 Univariate Analysis

In [ ]:
# Select numerical features (excluding 'datetime')
numerical_features = train_data.select_dtypes(include='number')

# Histograms
numerical_features.hist(figsize=(10, 10),bins=30)
plt.suptitle("Histograms")
plt.show()

# Boxplots
numerical_features.plot(kind='box', subplots=True, figsize=(10, 10),layout=(3, 3))
plt.suptitle("Box Plots")
plt.show()

### 3.3 Correlation analysis

In [ ]:
correlation = train_data.corr()['Totaal'].drop('datetime')
print(correlation.abs().sort_values(ascending=False))


# Correlation matrix
correlation_matrix = train_data.corr()
sns.heatmap(train_data.corr().drop(columns=['Totaal','datetime']), annot=True, cmap='YlGnBu')
plt.show()

- Low correlation with 'Totaal' in multiple variables - those will be removed.
- Strong multicollinearity between precipitation-rain and windspeed-windgusts.

In [ ]:
# Visualize the data
sns.pairplot(train_data)
plt.show()

## 4. Feature engineering

In [6]:
# changes prompted by exploration
train_data = train_data.drop(columns=['snow_depth (m)','windspeed_10m (km/h)','cloudcover (%)','precipitation (mm)','rain (mm)','snowfall (cm)'])
test_data = test_data.drop(columns=['snow_depth (m)','windspeed_10m (km/h)','cloudcover (%)','precipitation (mm)','rain (mm)','snowfall (cm)'])

In [7]:
# gentse feesten 2022
gentse_feesten_start_2022 = pd.to_datetime('2022-07-15')
gentse_feesten_end_2022 = pd.to_datetime('2022-07-24')

train_data['is_gentse_feesten_active'] = (
    (train_data['datetime'] >= gentse_feesten_start_2022) &
    (train_data['datetime'] <= gentse_feesten_end_2022)
).astype(int)


# Gentse feesten 2023
gentse_feesten_start_2023 = pd.to_datetime('2023-07-14')
gentse_feesten_end_2023 = pd.to_datetime('2023-07-23')

test_data['is_gentse_feesten_active'] = (
    (test_data['datetime'] >= gentse_feesten_start_2023) &
    (test_data['datetime'] <= gentse_feesten_end_2023)
).astype(int)

In [ ]:
## 1. train_data

# Hour of the day
train_data['hour'] = train_data['datetime'].dt.hour
train_data = pd.get_dummies(train_data, columns=['hour'], prefix='hour', prefix_sep='_')

# Day of the week
days_dummies = pd.get_dummies(train_data['datetime'].dt.dayofweek, prefix='day', prefix_sep='_')
train_data = pd.concat([train_data, days_dummies], axis=1)

# Months - dummy variables
months_dummies = pd.get_dummies(train_data['datetime'].dt.month, prefix='month')
train_data = pd.concat([train_data, months_dummies], axis=1)


In [ ]:
## 2. test_data

# Extract hours into dummy variables
test_data['hour'] = test_data['datetime'].dt.hour
test_data = pd.get_dummies(test_data, columns=['hour'], prefix='hour', prefix_sep='_')

# Day of the week
days_dummies = pd.get_dummies(test_data['datetime'].dt.dayofweek, prefix='day', prefix_sep='_')
test_data = pd.concat([test_data, days_dummies], axis=1)

# months:
months_dummies = pd.get_dummies(test_data['datetime'].dt.month, prefix='month')
test_data = pd.concat([test_data, months_dummies], axis=1)


missing_months = [column for column in set(train_data.columns) if column.startswith("month") and column not in set(test_data.columns)]
for column in missing_months:
    test_data[column] = False

- Adding weekend and summer vacation period is redundant as the variance is already explained by the days and months. It would result multicollinearity with months 7 and 8 and days 6 and 7.
- Only adding a variable for weekday vs weekend did not generate a better performance than using dummy variables for all days.
- Attempted capturing cyclic behavior of the hour of the day and day of the week by encoding as sin/cos values. This did not improve the model's performance.
- Added interaction terms to the model, this did not improve the model's performance either.

In [10]:
# drop datetime columns
train_data = train_data.drop('datetime', axis=1)
test_data = test_data.drop('datetime', axis=1)

## 5. Data exploration - revisited

In [21]:
correlation = train_data.corr()['Totaal']
print(correlation.abs().sort_values(ascending=False))

Totaal                      1.000000
hour_cos                    0.474936
hour_sin                    0.292305
day_sin                     0.252241
temperature_2m (°C)         0.222536
windgusts_10m (km/h)        0.075790
day_cos                     0.069820
inter                       0.026807
is_gentse_feesten_active    0.022801
Name: Totaal, dtype: float64


These new time-related variables show decent correlation with the number of cyclists and will be retained in the model.

## 6. Model

In [18]:
# Put data in numpy arrays
y = train_data.Totaal.values
X = train_data.drop('Totaal', axis=1).values

In [19]:
# to test on own data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7) # Use 70% of data for training

### 6.1 Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression


LinReg = LinearRegression() # call an instance of the class LinearRegression

LinReg.fit(X_train, y_train) # fit the model on the training data
y_hat_train = LinReg.predict(X_train) # predict training data
MSE_train = mean_squared_error(y_train, y_hat_train) # Compute training set MSE

y_hat_test = LinReg.predict(X_test) # predict test data
MSE_test = mean_squared_error(y_test, y_hat_test) # Test set MSE

R_train = LinReg.score(X_train, y_train) # Training set R²
R_test = LinReg.score(X_test, y_test) # Test set R²

print('Training set MSE: {}'.format(MSE_train))
print('Test set MSE: {}'.format(MSE_test))
print('Train set R²: {}'.format(R_train))
print('Test set R²: {}'.format(R_test))

### 6.2 Random Forest Regression

In [ ]:
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

r2 = r2_score(y_test, y_pred_rf)
print(f"R-squared: {r2}")

### 6.3 XGboost

In [ ]:

## gridsearch

param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    # Add other hyperparameters
}

grid_search = GridSearchCV(
    XGBRegressor(objective='reg:squarederror'),
    param_grid=param_grid,
    cv=5  # Number of cross-validation folds
)

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


In [ ]:

## random search
param_dist = {
    'learning_rate': uniform(0, 1),
    'n_estimators': randint(100, 1000),
    'max_depth': randint(1, 10),
    'min_child_weight': randint(1, 10),
    # Add other hyperparameters
}

random_search = RandomizedSearchCV(
    XGBRegressor(objective='reg:squarederror'),
    param_distributions=param_dist,
    n_iter=100,  # Number of random combinations to try
    cv=5
)

random_search.fit(X_train, y_train)
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

In [20]:
## for own data


# Initialize the XGBoost model
model = XGBRegressor(objective='reg:squarederror')

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")



Mean Squared Error: 19803.053024151526
R-squared: 0.7541775363584107


In [ ]:
## for kaggle data

# Initialize the XGBoost model
model = XGBRegressor(objective='reg:squarederror')

# Train the model on the training data
model.fit(X, y)

# Make predictions on the test set
y_pred = model.predict(test_data.drop("Id", axis=1))

## y_pred[y_pred < 0] = 0

# format predictions with Ids into dataframe and save to csv.
submission_file = pd.DataFrame([test_data["Id"], y_pred]).T
submission_file.columns = ["Id", "Predicted"]

submission_file.to_csv("submission.csv", index = False)


Attempts:
- linear regression
- polynomial terms
- random forest regression